<a href="https://colab.research.google.com/github/Ishaqinu/Data_science_portfolio/blob/main/Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np

loading training and testing data

In [4]:
df_train=pd.read_csv('/content/train.txt',sep=';',names=['comment','emotion'])
df_train.head()

,comment,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
df_train.emotion.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: emotion, dtype: int64

In [6]:
df_test=pd.read_csv('/content/test.txt',sep=';',names=['comment','emotion'])
df_test.head()

,comment,emotion
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [7]:
df_test.emotion.value_counts()

joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: emotion, dtype: int64

maping emotions

In [8]:
df_train['emotion_map']=df_train.emotion.map({'joy':0,'sadness':1,'anger':2,'fear':3,'love':4,'surprise':5})
df_test['emotion_map']=df_test.emotion.map({'joy':0,'sadness':1,'anger':2,'fear':3,'love':4,'surprise':5})
df_train.head()

,comment,emotion,emotion_map
0,i didnt feel humiliated,sadness,1
1,i can go from feeling so hopeless to so damned...,sadness,1
2,im grabbing a minute to post i feel greedy wrong,anger,2
3,i am ever feeling nostalgic about the fireplac...,love,4
4,i am feeling grouchy,anger,2


In [9]:
df_test.head()

,comment,emotion,emotion_map
0,im feeling rather rotten so im not very ambiti...,sadness,1
1,im updating my blog because i feel shitty,sadness,1
2,i never make her separate from me because i do...,sadness,1
3,i left with my bouquet of red and yellow tulip...,joy,0
4,i was feeling a little vain when i did this one,sadness,1


using Spacy

In [10]:
import spacy
nlp=spacy.load("en_core_web_sm")

def preprocess(text):
    list=[]
    for token in nlp(text):
        if not token.is_space and not token.is_punct:
            list.append(token.lemma_)
    return ' '.join(list)

In [12]:
df_train['preprocessed_comment']= df_train.comment.apply(preprocess)
df_test['preprocessed_comment']= df_test.comment.apply(preprocess)
df_train.preprocessed_comment.head()

0                              I do not feel humiliate
1    I can go from feel so hopeless to so damned ho...
2        I m grab a minute to post I feel greedy wrong
3    I be ever feel nostalgic about the fireplace I...
4                                    I be feel grouchy
Name: preprocessed_comment, dtype: object

In [13]:
df_test.head()

,comment,emotion,emotion_map,preprocessed_comment
0,im feeling rather rotten so im not very ambiti...,sadness,1,I m feel rather rotten so I m not very ambitio...
1,im updating my blog because i feel shitty,sadness,1,I m update my blog because I feel shitty
2,i never make her separate from me because i do...,sadness,1,I never make she separate from I because I don...
3,i left with my bouquet of red and yellow tulip...,joy,0,I leave with my bouquet of red and yellow tuli...
4,i was feeling a little vain when i did this one,sadness,1,I be feel a little vain when I do this one


Spliting input and out data from traing and testing data

In [14]:
x_train=df_train['preprocessed_comment']
y_train=df_train['emotion_map']
x_test=df_test['preprocessed_comment']
y_test=df_test['emotion_map']

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

pipeline , Tfidf Vectorizer , Random Foresr

In [16]:
pipe=Pipeline([('vectorize Tfidf',TfidfVectorizer()),('randomforest',RandomForestClassifier(n_estimators=250,criterion='gini'))])
pipe.fit(x_train,y_train)

Pipeline(steps=[('vectorize Tfidf', TfidfVectorizer()),
                ('randomforest', RandomForestClassifier(n_estimators=250))])

Predicting data

In [17]:
y_pred=pipe.predict(x_test)
y_pred

array([1, 1, 1, ..., 0, 0, 3])

In [21]:
from sklearn.metrics import classification_report,accuracy_score
print('classification report',classification_report(y_test,y_pred))

classification report               precision    recall  f1-score   support

           0       0.80      0.94      0.86       695
           1       0.91      0.88      0.89       581
           2       0.89      0.78      0.83       275
           3       0.83      0.80      0.81       224
           4       0.83      0.56      0.67       159
           5       0.59      0.48      0.53        66

    accuracy                           0.84      2000
   macro avg       0.81      0.74      0.77      2000
weighted avg       0.84      0.84      0.84      2000



In [23]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8405